# Combine CSVs

In [1]:
import polars as pl
import os

### Filepaths

In [7]:
bronze_path = os.path.join('..', 'data', 'bronze')
silver_path = os.path.join('..', 'data', 'silver')
nrdy_path = os.path.join('..', 'data', 'NRDY.csv')

In [55]:
# combine csvs
filenames = os.listdir(bronze_path)

dfs = []
for filename in filenames:
    filepath = os.path.join(bronze_path, filename)
    ticker = filename.removesuffix('.csv')

    df = pl.read_csv(filepath) \
        .select(
            pl.from_epoch(pl.col('timestamp'), time_unit="s").dt.date().alias("date"),
            pl.lit(ticker).alias('ticker'),
            pl.col('adjclose').alias('price')
        )
    
    dfs.append(df)

combined_df = pl.concat(dfs)

In [52]:
combined_path = os.path.join(silver_path, 'prices')

combined_df.write_parquet(combined_path)

### Preview

In [53]:
display(pl.read_parquet(combined_path))

date,ticker,price
date,str,f64
1994-03-18,"""SCSC""",1.71875
1994-03-21,"""SCSC""",1.90625
1994-03-22,"""SCSC""",1.9375
1994-03-23,"""SCSC""",2.09375
1994-03-24,"""SCSC""",2.09375
…,…,…
2024-09-09,"""HWBK""",23.612986
2024-09-10,"""HWBK""",23.612986
2024-09-11,"""HWBK""",23.870943


### Add NRDY

In [54]:
others_df = pl.read_parquet(combined_path)

nrdy_df = (pl.read_csv(nrdy_path)
    .select(
        pl.col('Date').str.to_date().alias('date'),
        pl.lit('NRDY').alias('ticker'),
        pl.col('Adj Close').alias('price')
    )
)
            

combined_with_nrdy_df = pl.concat([others_df, nrdy_df])

In [56]:
combined_with_nrdy_path = os.path.join(silver_path, 'prices_with_nrdy')

combined_with_nrdy_df.write_parquet(combined_with_nrdy_path)

### Preview

In [57]:
display(pl.read_parquet(combined_with_nrdy_path))

date,ticker,price
date,str,f64
1994-03-18,"""SCSC""",1.71875
1994-03-21,"""SCSC""",1.90625
1994-03-22,"""SCSC""",1.9375
1994-03-23,"""SCSC""",2.09375
1994-03-24,"""SCSC""",2.09375
…,…,…
2024-04-26,"""NRDY""",2.76
2024-04-29,"""NRDY""",2.71
2024-04-30,"""NRDY""",2.59
